In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cervical_Cancer"
cohort = "GSE114243"

# Input paths
in_trait_dir = "../../input/GEO/Cervical_Cancer"
in_cohort_dir = "../../input/GEO/Cervical_Cancer/GSE114243"

# Output paths
out_data_file = "../../output/preprocess/Cervical_Cancer/GSE114243.csv"
out_gene_data_file = "../../output/preprocess/Cervical_Cancer/gene_data/GSE114243.csv"
out_clinical_data_file = "../../output/preprocess/Cervical_Cancer/clinical_data/GSE114243.csv"
json_path = "../../output/preprocess/Cervical_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The role of DOCK10 in the regulation of the transcriptome and aging"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['strain: C57BL/6'], 1: ['genotype/variation: KO'], 2: ['treatment: Pre'], 3: ['sample: 6 to 10'], 4: ['cell type: spleen B cell'], 5: ['age: 12 weeks'], 6: ['reference composition: Spleen B cell pool of KO samples 6 to 10, before culture (Pre)']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this seems to be about human embryonic kidney 293T cells,
# which suggests gene expression data may be available, but we need to examine further.
# Since it's labeled as a SuperSeries composed of SubSeries, it's unclear if it contains gene expression data.
# To be conservative, we'll set is_gene_available to False.
is_gene_available = False

# 2. Variable Availability and Data Type Conversion
# From the sample characteristics dictionary, we only see tissue information.
# There's no information about cervical cancer, age, or gender.

# 2.1 Data Availability
trait_row = None  # No information about cervical cancer
age_row = None    # No age information
gender_row = None # No gender information

# 2.2 Data Type Conversion
# Define conversion functions even though we don't have the data
def convert_trait(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Convert to binary: 1 for cervical cancer, 0 for normal
    if 'cancer' in value.lower() or 'tumor' in value.lower() or 'carcinoma' in value.lower():
        return 1
    elif 'normal' in value.lower() or 'healthy' in value.lower() or 'control' in value.lower():
        return 0
    return None

def convert_age(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    # Try to extract age as a number
    try:
        # Extract digits if there are any
        import re
        digits = re.findall(r'\d+', value)
        if digits:
            return float(digits[0])
    except:
        pass
    return None

def convert_gender(value):
    if value is None:
        return None
    if ':' in value:
        value = value.split(':', 1)[1].strip().lower()
    # Convert to binary: 0 for female, 1 for male
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save initial filtering information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step since trait_row is None


A new JSON file was created at: ../../output/preprocess/Cervical_Cancer/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['(+)E1A_r60_1', '(+)E1A_r60_3', '(+)E1A_r60_a104', '(+)E1A_r60_a107',
       '(+)E1A_r60_a135', '(+)E1A_r60_a20', '(+)E1A_r60_a22', '(+)E1A_r60_a97',
       '(+)E1A_r60_n11', '(+)E1A_r60_n9', '(-)3xSLv1', 'A_51_P100034',
       'A_51_P100174', 'A_51_P100208', 'A_51_P100289', 'A_51_P100298',
       'A_51_P100309', 'A_51_P100327', 'A_51_P100347', 'A_51_P100519'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these are Agilent microarray probe IDs (starting with A_23_P),
# not standard human gene symbols. These probe IDs need to be mapped to official gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': ['400451', '10239', '9899', '348093', '57099'], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for mapping
# Based on the preview, the 'ID' column in gene_annotation contains probe IDs that match the 
# gene_data index, and 'GENE_SYMBOL' contains the gene symbols we want to map to
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'GENE_SYMBOL'  # Column containing gene symbols

# 2. Get the gene mapping dataframe by extracting the relevant columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert from probe-level to gene-level expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the first few gene symbols and their expression values
print("Gene expression data after mapping (first 5 genes):")
print(gene_data.index[:5])
print(gene_data.shape)


Gene expression data after mapping (first 5 genes):
Index(['A130033P14', 'A230055C15', 'A330041K01', 'A330044H09', 'A430017K17'], dtype='object', name='Gene')
(555, 40)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Create empty clinical features DataFrame since no trait data is available
clinical_features_df = pd.DataFrame()
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Create a minimal dataset for validation
minimal_df = pd.DataFrame({trait: [0, 1]})  # Dummy data with both classes represented

# Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # No trait data available
    is_biased=True,  # Consider biased since no trait data exists
    df=minimal_df,
    note="This dataset contains gene expression data but lacks information about cervical cancer status."
)

print("Data was determined to be unusable due to missing trait information and was not saved.")

Normalized gene data saved to ../../output/preprocess/Cervical_Cancer/gene_data/GSE114243.csv
Clinical data saved to ../../output/preprocess/Cervical_Cancer/clinical_data/GSE114243.csv
Abnormality detected in the cohort: GSE114243. Preprocessing failed.
Data was determined to be unusable due to missing trait information and was not saved.
